In [4]:
import os
os.chdir('/home/jupyter/fuck/aicup-meddata-pp_')

In [5]:
from openai import OpenAI
import openai
import tiktoken
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)  # for exponential backoff
from tqdm import tqdm
import time
import pandas as pd
api_key = "<your OpenAI api key>"
client = OpenAI(api_key=api_key)

In [3]:
def num_tokens_from_string(string: str, encoding_name: str = "cl100k_base") -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

# generate finetune file and create tuning job

In [ ]:
client.files.create(
  file=open('prompt1.jsonl', 'rb'),
  purpose="fine-tune"
)

In [ ]:
client.fine_tuning.jobs.create(
  training_file="<the file id from prompt1.jsonl>", 
  model="gpt-3.5-turbo",
  suffix="prompt1",
  hyperparameters={"n_epochs":10}
)

In [ ]:
client.files.create(
  file=open('prompt2.jsonl', 'rb'),
  purpose="fine-tune"
)

In [ ]:
client.fine_tuning.jobs.create(
  training_file="<the file id from prompt2.jsonl>", 
  model="gpt-3.5-turbo",
  suffix="prompt2",
  hyperparameters={"n_epochs":10}
)

In [ ]:
client.files.create(
  file=open('prompt3.jsonl', 'rb'),
  purpose="fine-tune"
)

In [ ]:
client.fine_tuning.jobs.create(
  training_file="<the file id from prompt3.jsonl>", 
  model="gpt-3.5-turbo",
  suffix="prompt3",
  hyperparameters={"n_epochs":10}
)

# predict

In [ ]:
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(3))
def label(record, sentence, model="gpt-3.5-turbo", prompt_type = 3):

    if is_SET_sentence(sentence):
        ths = {'twice':'R2', 'third':'R3', 'fourth':'R4', 'fifth':'R5', 'sixth':'R6', 'seventh':'R7', 'eighth':'R8', 'ninth':'R9', 'tenth':'R10'}
        response = []
        for key, value in ths.items():
            if key in sentence:
                response.append(f'SET:{key}=>{value}')
        response = '\\n'.join(response)
        return response
        
    elif len(sentence.lstrip().rstrip()) < 4:
        return 'PHI:Null'

    else:
        role = "You are a doctor, capable of identifying Protected Health Information (PHI) within medical records."

        if prompt_type == 1:
            phis = "PATIENT, DOCTOR, ROOM, DEPARTMENT, HOSPITAL, ORGANIZATION, STREET, CITY, STATE, COUNTRY, ZIP, LOCATION-OTHER, AGE, DATE, TIME, DURATION, SET, PHONE, URL, MEDICALRECORD, IDNUM"
            rule = f"PHI categories are: {phis}. If the content can not be identified as any of PHI category, reply a special string 'PHI:Null'"
            user_input = f"Please read the medical record: ```\n{record[:3659]}\n``` and identify all PHI categories in the the sentence: ```\n{sentence}\n```. Respond in the format 'PHI Category: PHI Content' for each identified PHI category. If there are multiple PHI categories in the sentence, separate all 'PHI Category: PHI Content' pairs with '\n'."

        elif prompt_type == 2:
            phis = "PATIENT, DOCTOR, DEPARTMENT, HOSPITAL, ORGANIZATION, STREET, CITY, STATE, COUNTRY, ZIP, AGE, DATE, TIME, MEDICALRECORD, IDNUM"
            rule = f"PHI categories are: {phis}. If the content can not be identified as any of PHI category, reply a special string 'PHI:Null'"
            user_input = f"Please read the medical record paragraph: ```\n{record}\n``` and identify all PHI categories in this sentence: ```\n{sentence}\n```. Respond in the format 'PHI Category: PHI Content' for each identified PHI category. If there are multiple PHI categories in the sentence, separate all 'PHI Category: PHI Content' pairs with '\n'."

        elif prompt_type ==3:
            phis = "PATIENT, DOCTOR, DEPARTMENT, HOSPITAL, ORGANIZATION, STREET, CITY, STATE, COUNTRY, ZIP, AGE, DATE, TIME, MEDICALRECORD, IDNUM"
            rule = f"PHI categories are: {phis}. If the content can not be identified as any of PHI category, reply a special string 'PHI:Null'"
            user_input = f"Please identify all PHI categories in this sentence (from a medical record): ```\n{sentence}\n```. Respond in the format 'PHI Category:PHI Content' for each identified PHI category. If multiple PHI categories are identified, concatenate 'PHI Category:PHI Content' pairs with '\n'."



        conversation = [
            {"role": "system", "content": role},
            {"role": "assistant", "content": rule},
            {"role": "user", "content": user_input}
        ]

        tokens_remain = 4096 - num_tokens_from_string(user_input)

        response = client.chat.completions.create(
            model=model,
            messages=conversation,
            max_tokens=(256 if tokens_remain >= 256 else tokens_remain)
        )


        r = response.choices[0].message.content

        if 'SET:' in r:
            r = 'PHI:Null'

        elif 'AGE:' in r:
            if not r.replace('AGE:','').isdigit():
                r = 'PHI:Null'

        elif len(r) < 1:
            r = 'PHI:Null'
        elif 'PHI:' in r and len(r.replace('PHI:','')) < 2:
            r = 'PHI:Null'

        return r



def is_SET_sentence(sentence):
    import re
    pattern1 = r'\b(?:examined|repeated|tested|re-examined|reexamined|evaluated|assessed|verified|scrutinized|proven|inspected|surveyed|investigated|analyzed|reviewed|reassessed|reevaluated|retested|reworked|iterated|recurred|replicated|duplicated|recycled)\b\s\b(?:twice|third|fourth|fifth|sixth|seventh|eighth|ninth|tenth)\b'
    pattern2 = r'\b(?:examined|repeated|tested|re-examined|reexamined|evaluated|assessed|verified|scrutinized|proven|inspected|surveyed|investigated|analyzed|reviewed|reassessed|reevaluated|retested|reworked|iterated|recurred|replicated|duplicated|recycled)\b\s\(\b(?:twice|third|fourth|fifth|sixth|seventh|eighth|ninth|tenth)\b\)'

    match = re.search(pattern1, sentence, re.IGNORECASE) or re.search(pattern2, sentence, re.IGNORECASE)

    return True if match else False

In [ ]:
import pickle as pkl
import time
import random

valid = pd.read_csv('./test_pp_context.tsv', delimiter='\t').fillna('')

model, prompt_type, output_file = '<the model id fintuned by prompt1.jsonl>', 1, 'prompt2_result'
model, prompt_type, output_file = '<the model id fintuned by prompt2.jsonl>', 2, 'prompt2_result'
model, prompt_type, output_file = '<the model id fintuned by prompt3.jsonl>', 3, 'prompt3_result'

labels = []


i = 0
for id, row in tqdm(valid.iterrows()):
    file, sentence = row.file, row.sentence
    record_path = './content/First_Phase_Release/Validation_Release'
    if prompt_type == 1:
        record = open(f'{record_path}/{file}.txt', 'r').read()
    else:
        record = row.context
    response = label(record, sentence, model, prompt_type)
    labels.append(response)
    i += 1

valid['answer'] = labels     
print('labels finish! len=', len(labels))


files = []
phis = []
start_ids = []
end_ids = []
contents = []
for id, row in valid.iterrows():
    file, start_id, sentence, answer = row.file, row.start_id, row.sentence, row.answer
    for ans in answer.split('\\n'):
        if ans == 'PHI:Null':
            continue

        phi = ans.split(':')[0]
        content = ':'.join(ans.split(':')[1:])
        content = content.lstrip().rstrip()

        if content == 'Null' or (not content.isdigit() and len(content)==1):
            continue

        if phi == 'AGE' and (content.isdigit() and not 0<int(content) <100):
            continue


        content2=''
        if phi in ["DATE", "TIME", "DURATION", "SET"]:
            content_tmp = content
            content = content_tmp.split('=>')[0]
            content2 = ''.join(content_tmp.split('=>')[1:])

        anchor = start_id
        offset = sentence.find(content)
        start_id = start_id+offset
        end_id = start_id + len(content)

        content = content+'<<<SEP>>>'+content2 if content2 else content

        if len(content) > 0 and phi in ["PATIENT", "DOCTOR", "USERNAME", "PROFESSION", "ROOM", "DEPARTMENT", "HOSPITAL", "ORGANIZATION", "STREET", "CITY", "STATE", "COUNTRY", "ZIP", "LOCATION-OTHER", "AGE", "DATE", "TIME", "DURATION", "SET", "PHONE", "FAX", "EMAIL", "URL", "IPADDR", "SSN", "MEDICALRECORD", "HEALTHPLAN", "ACCOUNT", "LICENSE", "VEHICLE", "DEVICE", "BIOID", "IDNUM"]:
            files.append(file)
            phis.append(phi)
            start_ids.append(start_id)
            end_ids.append(end_id)
            contents.append(content)
        start_id = end_id
        sentence = sentence[offset+len(content):]
        
        

answers = pd.DataFrame({'file':files, 'phi':phis, 'start_id':start_ids, 'end_id':end_ids, 'content':contents})
answers.to_csv(output_file, header=False, index=False, sep='\t')

with open(output_file, 'r') as r, open(output_file+'.out', 'w') as w:
    for rline in r:
        if rline.split('\t')[1] in ["DATE", "TIME", "DURATION", "SET"]:
            rline = rline.replace('<<<SEP>>>', '\t')
        w.write(rline)

    